In [2]:
import torch
from torch.utils.data import Dataset, DataLoader
import os, os.path 
import numpy 
import pickle
from glob import glob

"""Change to the data folder"""
new_path = "./new_train/new_train"

# number of sequences in each dataset
# train:205942  val:3200 test: 36272 
# sequences sampled at 10HZ rate

### Create a dataset class 

In [3]:
class ArgoverseDataset(Dataset):
    """Dataset class for Argoverse"""
    def __init__(self, data_path: str, transform=None):
        super(ArgoverseDataset, self).__init__()
        self.data_path = data_path
        self.transform = transform

        self.pkl_list = glob(os.path.join(self.data_path, '*'))
        self.pkl_list.sort()
        
    def __len__(self):
        return len(self.pkl_list)

    def __getitem__(self, idx):

        pkl_path = self.pkl_list[idx]
        with open(pkl_path, 'rb') as f:
            data = pickle.load(f)
            
        if self.transform:
            data = self.transform(data)

        return data


# intialize a dataset
val_dataset  = ArgoverseDataset(data_path=new_path)

### Create a loader to enable batch processing

In [4]:
batch_sz = 64

def my_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    out = [numpy.dstack([scene['p_out'], scene['v_out']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    out = torch.LongTensor(out)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, out, scene_ids, track_ids, agent_ids]

def test_collate(batch):
    """ collate lists of samples into batches, create [ batch_sz x agent_sz x seq_len x feature] """
    inp = [numpy.dstack([scene['p_in'], scene['v_in']]) for scene in batch]
    scene_ids = [scene['scene_idx'] for scene in batch]
    track_ids = [scene['track_id'] for scene in batch]
    agent_ids = [scene['agent_id'] for scene in batch]
    inp = torch.LongTensor(inp)
    scene_ids = torch.LongTensor(scene_ids)
    return [inp, scene_ids, track_ids, agent_ids]

val_loader = DataLoader(val_dataset,batch_size=batch_sz, shuffle = False, collate_fn=my_collate, num_workers=0)

### Visualize the batch of sequences

In [12]:
import matplotlib.pyplot as plt
import random
from tqdm.notebook import tqdm, trange

agent_id = 0
epoch = 6

def show_sample_batch(sample_batch, agent_id):
    """visualize the trajectory for a batch of samples with a randon agent"""
    inp, out, scene_ids, track_ids, agent_ids = sample_batch
    batch_sz = inp.size(0)
    agent_sz = inp.size(1)
    
    fig, axs = plt.subplots(1,batch_sz, figsize=(15, 3), facecolor='w', edgecolor='k')
    fig.subplots_adjust(hspace = .5, wspace=.001)
    axs = axs.ravel()   
    for i in range(batch_sz):
        axs[i].xaxis.set_ticks([])
        axs[i].yaxis.set_ticks([])
        
        # first two feature dimensions are (x,y) positions
        axs[i].scatter(inp[i, agent_id,:,0], inp[i, agent_id,:,1])
        axs[i].scatter(out[i, agent_id,:,0], out[i, agent_id,:,1])

cuda_status = torch.cuda.is_available()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = torch.nn.Sequential(
    torch.nn.Linear(76, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 98),
    torch.nn.ReLU(),
    torch.nn.Linear(98, 120)
)
model.to(device)
model = model.cuda()

# Use the nn package to define our loss function
loss_fn=torch.nn.MSELoss()

# Use the optim package to define an Optimizer

learning_rate =1e-3
optimizer = torch.optim.RMSprop(model.parameters(), lr=learning_rate)

for i in trange(epoch):
    
    for i_batch, sample_batch in enumerate(tqdm(val_loader)):
        inp, out, scene_ids, track_ids, agent_ids = sample_batch
        """TODO:
          Deep learning model
          training routine
        """
        x = torch.flatten(inp, start_dim=2)

        x = x.float()
        y = out.float()

        if cuda_status:
            #model = model.cuda()
            #x = inp.cuda()
            #y = out.cuda()
            x.to(device)
            y.to(device)
            x = x.cuda()
            y = y.cuda()

        y_pred = None

        #for t in tqdm(range(100), desc='Interation'):

        # Forward pass: predict y by passing x to the model.    
        y_pred = model(x)
        y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))

        # Compute the loss.
        loss = loss_fn(y_pred, y)
        print(torch.sqrt(loss))

        # Before backward pass, zero outgradients to clear buffers  
        optimizer.zero_grad()

        # Backward pass: compute gradient w.r.t modelparameters
        loss.backward()

        # makes an gradient descent step to update its parameters
        optimizer.step()

        #print(y_pred.shape)
        #show_sample_batch(sample_batch, agent_id)

        if i_batch == 3216:
            #show_sample_batch(sample_batch, agent_id)
            #show_sample_batch([inp, y_pred.cpu().detach(), scene_ids, track_ids, agent_ids], agent_id)
            break


tensor(596.8904, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(559.6510, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(397.2090, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(708.8192, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(397.9312, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(361.5768, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(268.1245, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(278.8512, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(283.3221, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(248.0403, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(265.5246, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(233.8657, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(257.5554, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(246.0152, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(245.0039, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(228.7114, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(203.7533, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(213.634

tensor(25.0753, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(28.9525, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(27.0026, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(26.8691, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(30.2848, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.6021, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.5064, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.4104, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.0939, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.2313, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(24.4012, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.8236, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(33.4654, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(37.5054, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(43.3344, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(43.5236, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.9028, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.9154, device='cuda:0'

tensor(48.1789, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(58.6858, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.9843, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(81.0525, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(84.8232, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(100.8181, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(38.1169, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.7341, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.7143, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(34.9120, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.5259, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(23.1138, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.5350, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.0551, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.4021, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.4244, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.8834, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.5759, device='cuda:0

tensor(19.5838, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.9591, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.9355, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.6423, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(29.1684, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(35.1153, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(41.1461, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(52.1050, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(48.0448, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.0371, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(46.3530, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(65.9199, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(72.5160, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(82.3955, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(69.0966, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(74.7764, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(63.7527, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(60.9988, device='cuda:0'

tensor(25.4996, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.9443, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(18.3310, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.0584, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.2740, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.8164, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.4641, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.9506, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.1166, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.4474, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.7909, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(25.8503, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(32.3765, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(30.7394, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.6627, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.5099, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.1206, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(17.7528, device='cuda:0'

...

tensor(5.9770, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.0495, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.0178, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.8185, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.4938, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(16.6559, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.3511, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.5119, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(15.0348, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.5307, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.9554, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.5951, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.3916, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.0326, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.5031, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.2183, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.4045, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.9653, device='cuda:0', gra

tensor(26.6319, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(24.5667, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(24.2371, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(21.2046, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(22.1522, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.0666, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(9.9696, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.2526, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.1701, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.6970, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.0923, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.5819, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.9844, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(8.4060, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.3327, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.9949, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.7415, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.8352, device='cuda:0', grad_f

tensor(11.9405, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.7163, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.0264, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.1048, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.5868, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.2367, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.8756, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6824, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6511, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.0722, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.1320, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.9772, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.4897, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(4.8320, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.2467, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(5.5138, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(6.3442, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(7.0236, device='cuda:0', grad_fn=<SqrtBa

tensor(23.5501, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(20.9709, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(19.4206, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.2909, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.7324, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.9050, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.1652, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.9741, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.6364, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.1841, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.4792, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.6397, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.3832, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(10.9109, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(11.5022, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(12.8129, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(14.0757, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(13.9570, device='cuda:0'

tensor(3.0727, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.3318, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(3.1735, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.8869, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6375, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.6926, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.4390, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.2142, device='cuda:0', grad_fn=<SqrtBackward>)
tensor(2.2451, device='cuda:0', grad_fn=<SqrtBackward>)




In [14]:
torch.save(model, './6epochmodel.pt')

In [5]:
model = torch.load('./6epochmodel.pt')
model.eval()

Sequential(
  (0): Linear(in_features=76, out_features=98, bias=True)
  (1): ReLU()
  (2): Linear(in_features=98, out_features=98, bias=True)
  (3): ReLU()
  (4): Linear(in_features=98, out_features=98, bias=True)
  (5): ReLU()
  (6): Linear(in_features=98, out_features=120, bias=True)
)

In [7]:
import pandas as pd
from tqdm.notebook import tqdm, trange

# Submission output
writeCSV = True
val_path = "./new_val_in/new_val_in"

if writeCSV:
    
    dataset = ArgoverseDataset(data_path=val_path)
    test_loader = DataLoader(dataset,batch_size=batch_sz, shuffle = False, collate_fn=test_collate, num_workers=0)
    
    data = []
    
    with torch.no_grad():
        for i_batch, sample_batch in enumerate(tqdm(test_loader)):
            inp, scene_ids, track_ids, agent_ids = sample_batch
            inp = torch.flatten(inp, start_dim=2)

            if cuda_status:
                model = model.cuda()
                x = inp.cuda()
            else:
                x = inp

            y_pred = None

            # Forward pass: predict y by passing x to the model.    
            y_pred = model(x.float())
            y_pred = torch.reshape(y_pred, torch.Size([batch_sz, 60, 30, 4]))
            
            #print(track_ids[63][15][0].item())
            #for i in range(30):
            #    print(track_ids[15][i][0].item())
            #print(agent_ids[15])
            #break
            
            #vehicle_index = 0
            #found = False
            #while not found:
            #    if track_ids[i][vehicle_index][j][0] == agent_id:
            #        found = True
            #    else:
            #        vehicle_index += 1
            
            for i in range(batch_sz):
                row = []
                row.append(scene_ids[i].item())
                curr = y_pred[i]
                #for j in range(60):
                #    row.append(str(curr[j][29][0].item()) + ' ' + str(curr[j][29][1].item()))
                #data.append(row)
                
                agent_id = agent_ids[i]
                
                for j in range(30):
                    vehicle_index = 0
                    found = False
                    while not found:
                        if track_ids[i][vehicle_index][j][0] == agent_id:
                            found = True
                        else:
                            vehicle_index += 1

                    row.append(str(curr[vehicle_index][j][0].item()))
                    row.append(str(curr[vehicle_index][j][1].item()))
                    
                data.append(row)

    df = pd.DataFrame(data, columns = ['ID','v1','v2','v3','v4','v5','v6','v7','v8','v9','v10','v11','v12','v13','v14','v15','v16','v17','v18','v19','v20','v21','v22','v23','v24','v25','v26','v27','v28','v29','v30','v31','v32','v33','v34','v35','v36','v37','v38','v39','v40','v41','v42','v43','v44','v45','v46','v47','v48','v49','v50','v51','v52','v53','v54','v55','v56','v57','v58','v59','v60'])
    print(df)
    df.to_csv('submission.csv', index=False)
                
                
                

NameError: name 'cuda_status' is not defined